In [ ]:
## Extracting the data and saving it as a CSV

In [1]:
import sys
sys.path.append('../sentiment_analysis')

# This is another way of doing the above, not sure which is better
sys.path.insert(0, '../mining')

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vader_sentiment_analysis import SentimentAnalyser
import pandas as pd

import connector
import query

/home/alexander/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df = pd.read_csv("../data/SELECT_ALL_FROM_all_messages.csv", header = 0)

/home/sophie/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
len(df)

1354178

In [16]:
list(df)

['Unnamed: 0',
 'user_id',
 'contact_id',
 'relationship',
 'channel',
 'sent_at',
 'message']

In [8]:
df_meta = pd.read_csv("../data/SELECT_ALL_FROM_all_messages_metadata.csv", header = 0)

In [9]:
len(df_meta)

1070395

In [4]:
# Not able to read this in. 
#df = pd.read_csv("../data/all_messages_cleaned.csv", encoding='utf8', engine='python', usecols = ['user_id'], header = 0)

In [10]:
df = df.drop(['Unnamed: 0'],axis=1)

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

port = 5432
remote_bind_address = ('localhost', 5432)
local_bind_address = ('localhost', 5432)
user_name = 'analytics'
dbname = 'dreikanter_production'
remote_host = 'dreikanter.production.devguru.co'
remote_port = 22
conn = None
server = None


params = {
            'database': 'dreikanter_production',
            'user': 'analytics',
            'host': 'localhost',
            'port': 5432
        }

server = SSHTunnelForwarder((remote_host, remote_port),ssh_username=user_name,
                                    local_bind_address=local_bind_address,
                                    remote_bind_address=remote_bind_address)
server.start()
print "We are in!"

#"dbname=dreikanter_production user=analytics host = localhost port = 5432"
conn = psycopg2.connect(**params)

We are in!


In [20]:
conn.close()
server.stop()

In [80]:

sql_query = """SELECT * FROM all_msgs_tf;"""

dbconnection = connector.ConnectDB()
conn = dbconnection.get_connection()
q = query.Query(conn, sql_query)
query_list = q.get_query_list()
query_df = q.get_query_dataframe()
#q.write_df_to_csv()

#query_df['body'][6:16]
#query_df.head(22)

2016-08-15 16:18:49,082 | ERROR   | Couldn't open tunnel localhost:5432 <> localhost:5432 might be in use or destination not reachable.


In [81]:
query_df.shape

(2859488, 7)

In [94]:
df_no_email = query_df[query_df['channel'] != 'gmail']

In [83]:
df_no_email.shape
#print query_df[query_df.message.str.contains("Land Rover")]

(2629679, 7)

In [117]:
df_no_email = df_no_email[df_no_email['message'] != '        ']

In [118]:
df_no_email.shape

(2517804, 7)

In [119]:
b = [len(str(x).split()) for x in df_no_email['message']]

In [125]:
df_no_email['word_count'] = b

In [167]:
df_no_email[''].isnull().sum()

292013

In [174]:
df_no_email[df_no_email['positive'].isnull()]

,user_id,contact_id,relationship,channel,sent_at,message,to_from,word_count,positive,negative,neutral,compound
2517804,11338,25888,Partner,text_message,2015-11-27 23:39:05,Weren't exactly dating?,True,3,NaN,NaN,NaN,NaN
2517805,11338,25888,Partner,text_message,2015-11-27 23:39:34,I'm not allowed to date...,False,5,NaN,NaN,NaN,NaN
2517806,11338,25888,Partner,text_message,2015-11-27 23:40:07,Really were u doing what we are doing,True,8,NaN,NaN,NaN,NaN
2517807,11338,25888,Partner,text_message,2015-11-27 23:40:47,Sorta ya,False,2,NaN,NaN,NaN,NaN
2517808,11338,25888,Partner,text_message,2015-11-27 23:41:23,Um,True,1,NaN,NaN,NaN,NaN
2517809,11338,25888,Partner,text_message,2015-11-27 23:41:37,What,False,1,NaN,NaN,NaN,NaN
2517810,11338,25888,Partner,text_message,2015-11-27 23:41:54,Nothing,True,1,NaN,NaN,NaN,NaN
2517811,11338,25888,Partner,text_message,2015-11-27 23:42:24,No tell me,False,3,NaN,NaN,NaN,NaN
2517812,11338,25888,Partner,text_message,2015-11-27 23:42:50,Trust me nothing ur turn,True,5,NaN,NaN,NaN,NaN
2517813,11338,25888,Partner,text_message,2015-11-27 23:44:38,Ok umm...,False,2,NaN,NaN,NaN,NaN


In [130]:
sentiment_analyser = SentimentAnalyser()
result = sentiment_analyser.run_vader(df_no_email, 'message')

In [175]:
result.head()

,user_id,contact_id,relationship,channel,sent_at,message,to_from,word_count,positive,negative,neutral,compound
31,10,35,Ex,text_message,2011-02-18 06:50:35,Ill be there in 3,False,5,0.106,0.243,0.652,-0.1725
32,10,35,Ex,text_message,2011-02-18 06:50:47,hey I'm in the theater,False,5,0.280,0.164,0.556,0.5788
35,10,35,Ex,text_message,2011-02-22 02:22:54,:),False,1,0.727,0.000,0.273,0.7906
37,10,35,Ex,text_message,2011-03-04 07:08:49,Hey! How was the trip?,False,5,0.480,0.000,0.520,0.9399
38,10,35,Ex,text_message,2011-03-04 08:17:59,"great! still feels like I'm recovering, though!",False,7,0.129,0.000,0.871,0.5487


In [179]:
result.to_csv('all_messages_no_gmail_tf_wc_sentiment.csv', encoding='utf-8', index=False)

In [180]:
df = pd.read_csv('./all_messages_no_gmail_tf_wc_sentiment.csv', header=0)

In [181]:
df.head()

,user_id,contact_id,relationship,channel,sent_at,message,to_from,word_count,positive,negative,neutral,compound
0,10,35,Ex,text_message,2011-02-18 06:50:35,Ill be there in 3,False,5,0.106,0.243,0.652,-0.1725
1,10,35,Ex,text_message,2011-02-18 06:50:47,hey I'm in the theater,False,5,0.280,0.164,0.556,0.5788
2,10,35,Ex,text_message,2011-02-22 02:22:54,:),False,1,0.727,0.000,0.273,0.7906
3,10,35,Ex,text_message,2011-03-04 07:08:49,Hey! How was the trip?,False,5,0.480,0.000,0.520,0.9399
4,10,35,Ex,text_message,2011-03-04 08:17:59,"great! still feels like I'm recovering, though!",False,7,0.129,0.000,0.871,0.5487
